In [11]:
import torch
import torchvision
import numpy as np
import gradient_reversal
import joblib
import os

In [12]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


# Import WIKI_EVE

In [13]:
import wiki_eve_max.model as models
import wiki_eve_max.loss as loss
import wiki_eve_max.transform as transform
import wiki_eve_max.dataset as dataset
import wiki_eve_max.work as work

In [18]:
CWD = os.getcwd()
META_FILE = os.path.join(CWD, "DATASETS_TEST/NP/even_spacing/mean/32", "metaALL0-6_static.dump")
DATASET = dataset.BFIKEYSTROKE(
    meta_file=META_FILE,
    transform=torchvision.transforms.Compose([transform.Absolute()])
)

In [23]:
print(DATASET.m_len)
print(DATASET.m_domains)
print(DATASET.m_num_domain)


184000
['0000', '0001', '0002', '0003', '0004', '0005', '0006', '0007', '0008', '0009', '0010', '0011', '0012', '0013', '0014', '0015', '0016', '0017', '0018', '0019', '0020', '0021', '0022', '0023', '0024', '0025', '0026', '0027', '0028', '0029', '0030', '0031', '0032', '0033', '0034', '0035', '0036', '0037', '0038', '0039', '0040', '0041', '0042', '0043', '0044', '0045', '0046', '0047', '0048', '0049', '0050', '0051', '0052', '0053', '0054', '0055', '0056', '0057', '0058', '0059', '0060', '0061', '0062', '0063', '0064', '0065', '0066', '0067', '0068', '0069', '0070', '0071', '0072', '0073', '0074', '0075', '0076', '0077', '0078', '0079', '0080', '0081', '0082', '0083', '0084', '0085', '0086', '0087', '0088', '0089', '0090', '0091', '0092', '0093', '0094', '0095', '0096', '0097', '0098', '0099', '0100', '01000', '01001', '01002', '01003', '01004', '01005', '01006', '01007', '01008', '01009', '0101', '01010', '01011', '01012', '01013', '01014', '01015', '01016', '01017', '01018', '0101

In [32]:
print(DATASET.m_domains[DATASET[183000]["domain"]])

6900


In [6]:
DATASET[0]["data"].shape

AttributeError: 'BFIKEYSTROKE' object has no attribute 'm_num_class'

In [7]:
print(len(DATASET),len(DATASET.m_domains),DATASET.m_num_domain, DATASET.m_num_class)

184000 18400 18400 10


In [8]:
train_data, test_data = DATASET.split(train_p=0.90, total_samples=40000)
print(train_data[9])
print(train_data[25])

{'data': tensor([[0.4821, 0.5534, 0.5321,  ..., 0.7551, 0.3716, 0.3609],
        [0.4464, 0.6514, 0.5374,  ..., 0.4991, 0.3091, 0.5317],
        [0.2430, 0.4934, 0.5111,  ..., 0.2430, 0.4742, 0.8057],
        ...,
        [0.6514, 0.5320, 0.3637,  ..., 0.7323, 0.4148, 0.5584],
        [0.4714, 0.4517, 0.6447,  ..., 0.8600, 0.4273, 0.1421],
        [0.5080, 0.3369, 0.6936,  ..., 0.3171, 0.3369, 0.5683]]), 'label': 6, 'domain': 10782, 'domain_0': 9038}
{'data': tensor([[0.1001, 0.6476, 0.6270,  ..., 0.3879, 0.5559, 0.4286],
        [0.4555, 0.5982, 0.5516,  ..., 0.1010, 0.1395, 0.7545],
        [0.3369, 0.3117, 0.1497,  ..., 0.8282, 0.5047, 0.4396],
        ...,
        [0.5954, 0.3533, 0.2835,  ..., 0.2456, 0.7730, 0.3376],
        [0.5260, 0.7329, 0.7467,  ..., 0.1036, 0.2977, 0.2120],
        [0.4214, 0.0735, 0.0489,  ..., 0.8783, 0.3115, 0.3029]]), 'label': 3, 'domain': 12068, 'domain_0': 15204}


In [9]:
torch.cuda.empty_cache()

SC*NR*NC

In [10]:
234*4*2

1872

In [11]:
learning_rate = 1e-3
batch_size = 10
epochs = 10

train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)
channels = 234*4*2

model = models.WIKI_EVE_1D(
    conv_in_channels = channels,
    conv_out_channels = channels,
    conv_kernel_size = 16,
    conv_stride = 1,
    GF_output_size = 150,
    GC_labels = 10,
    GD_labels = 2,#DATASET.m_num_domain,
    groups=channels,
    lambda_domain = -0.5
)
print(model)
model.to(torch.device(device=device))
loss_fn = loss.AdvLernLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    work.train_loop(train_dataloader, model, loss_fn, optimizer, batch_size=batch_size, device=device, v=False)
    work.test_loop(test_dataloader, model, loss_fn, device=device, v=False)
print("Done!")

WIKI_EVE_1D(
  (GF): Sequential(
    (0): Conv1d(1872, 1872, kernel_size=(16,), stride=(1,), groups=1872)
    (1): ReLU()
    (2): AdaptiveMaxPool1d(output_size=450)
    (3): Conv1d(1872, 1872, kernel_size=(16,), stride=(1,), groups=1872)
    (4): ReLU()
    (5): AdaptiveMaxPool1d(output_size=150)
  )
  (GC): Sequential(
    (0): Linear(in_features=280800, out_features=1000, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1000, out_features=1000, bias=True)
    (3): ReLU()
    (4): Linear(in_features=1000, out_features=10, bias=True)
  )
  (GD): Sequential(
    (0): Linear(in_features=280800, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=2, bias=True)
    (3): Sigmoid()
  )
)
Epoch 1
-------------------------------
loss: 2.099333  [   10/36000]
loss: 56.661297  [   20/36000]
loss: 12.694902  [   30/36000]
loss: 8.273081  [   40/36000]
loss: 9.799745  [   50/36000]
loss: 14.629787  [   60/36000]
loss: 4.271248  [   70/36000]
loss: 5.63

In [13]:
work.test_loop(test_dataloader, model, loss_fn, device=device, v=False)


* Test Error: 
 Accuracy (label): 100.0%, Accuracy (domain): 0.0%, Avg loss: -0.393684 

